# 1-Retrieve data

import necesaary libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

import os, sys, pickle, copy, pygmt, pooch, shutil


from pathlib import Path

import pandas as pd

import numpy as np

from matplotlib import pyplot as plt
from scipy import spatial

import xarray as xr

import pyproj as proj

from zipfile import ZipFile

from time import sleep
from tqdm import tqdm, tqdm_notebook

from osgeo import gdal

import AFQ_utils as utils


In [3]:
# constanst
crs_to='epsg:4326'
crs_from='epsg:4326'
projection = 'M5.4i'
#parent directory

DIR = Path().resolve() 

#constants
km = 1000


# We can exclude Arctic ocean and Antarctica, as there are no HF measurements to use
world_lon_min, world_lon_max, world_lat_min, world_lat_max = -180, 180, -60, 80

# map extents of Africa
afr_lon_min, afr_lon_max, afr_lat_min, afr_lat_max =  -20, 52, -37 , 38  


region_afr = [afr_lon_min, afr_lon_max, afr_lat_min, afr_lat_max]
region_world = [world_lon_min, world_lon_max, world_lat_min, world_lat_max]

In [4]:
# create necessray folders

path = os.path.join(DIR, 'Fig')
Path(path).mkdir(parents=True, exist_ok=True)

path = os.path.join(DIR, 'Grids')
Path(path).mkdir(parents=True, exist_ok=True)

path = os.path.join(DIR, 'Grids', 'Inputs')
Path(path).mkdir(parents=True, exist_ok=True)

path = os.path.join(DIR, 'Grids', 'Outputs')
Path(path).mkdir(parents=True, exist_ok=True)

path = os.path.join(DIR, 'Hyperparameters')
Path(path).mkdir(parents=True, exist_ok=True)


path = os.path.join(DIR, 'KPI')
Path(path).mkdir(parents=True, exist_ok=True)


path = os.path.join(DIR, 'Dataset')
Path(path).mkdir(parents=True, exist_ok=True)

path = os.path.join(DIR, 'Dataset', 'Preprocessed')
Path(path).mkdir(parents=True, exist_ok=True)

path = os.path.join(DIR, 'Dataset', 'References')
Path(path).mkdir(parents=True, exist_ok=True)


references = ['a_Moho_Depth', 'b_LAB', 'c_Distance_to_Nearest_Volcano', 'd_SV', 'e_PV',
              'f_CTD', 'g_Magnetic', 'h_DEM', 'i_Crustal_Rho', 'i_Lithospheric_Rho', 'k_Free_Air',
              'l_Geoid', 'm_Bouguer','n_Shape_Index', 'o_Tectonic_Regionalization', 
              'p_GLiM', 'q_Heat_Flow']

for reference in references :
    path = os.path.join(DIR, 'Dataset', 'References', reference)
    Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
obs = pd.DataFrame()


obs["OBS_REF"] = ["CTD" ,  "SI","LAB", "MOHO",
            "SV","PV", 
            "GEOID","FA","DEM","BG", "EMAG2_CLASS",
                   "RHO_L", "RHO_C", 
                  "VOLC_DIST_W", "REG", "GLIM"]

obs["OBS_AFR"] = ["CTD" ,  "SI","LAB", "MOHO",
            "SV_SPEED","PV_SPEED", 
            "GEOID","FA","DEM","BG", "EMAG2",
                   "RHO_L", "RHO_C", 
                  "VOLC_DIST", "REG", "GLIM"]
  
     
# Labels for plots etc


# Labels for plots etc
obs["LABELS_gmt"] = ["CTD",  "Shape index", "LAB", "Moho", 
                "S@_v@ 150km", "P@_v@ 150km", 
                "Geoid", "Free air", "DEM", "Bouguer", "Mag.", 
                "Lith. rho", "Crust rho",  
                 "Volcano", "REG", "GliM", ]  


obs["LABELS"] = ["CTD",  "Shape index", "LAB", "Moho", 
                "$S_v$ @150km", "$P_v$ @150km", 
                "Geoid", "Free air", "DEM", "Bouguer", "Mag.", 
                "Lith. ρ", "Crust ρ",  
                 "Volcano", "REG", "GliM", ]
    
# "vp/vs"
# Units to display in plots etc
obs["UNITS"] = ["km",  "si", "km", "km",
             "$\delta$$S_v$ %","$\delta$$P_v$ %", 
             "m", "mGal", "m", "mGal",  "f(nT)", 
                 "kg/m$^3$", "kg/m$^3$",
                "km",  "class", "class"]



obs["UNITS_gmt"] = ["km",  "si", "km", "km",
             "km/s","km/s", 
             "m", "mGal", "m", "mGal",  "f(nT)", 
                 "kg/m@+3@+", "kg/m@+3@+",
                "km",  "class", "class"]
        
# Range of colormap for plots. Similar data are placed in same ranges for consistancy
obs["V_RANGE"] = [(0,50), (-1,1),(0,300),(15,60),
              (-0.075,0.075), (-0.02,0.02), 
              (-45,45), (-100,100) , (-2200, 2200),(-100,100),  (-0.4, 0.4), 
                   (3260, 3360), (2650, 2950),
                  (0,1), (1,6),(1,16),]


    
obs["V_RANGE_AFR"] = [(0,50), (-1,1),(50,250),(20,50),
          (-0.075,0.075), (-0.02,0.02), 
          (-45,45), (-100,100) , (-2200, 2200),(-100,100),  (-200, 200), 
               (3260, 3360), (2650, 2950),
              (0,100), (1,6),(1,15),]


obs["CMAPS"] = ["batlow",  "broc", "bamako", "batlow", 
             "roma","roma", 
             "bamako", "broc", "bukavu", "broc", "batlow",            
                "batlow", "batlow",
               "bamako",  "batlowS","categorical", ]

obs["CMAPS"] = ["SCM/bamako",  "SCM/broc", "SCM/bamako", "SCM/bamako", 
             "SCM/roma","SCM/roma", 
             "SCM/bamako", "SCM/broc", "SCM/oleron", "SCM/broc", "SCM/bilbao",            
                "SCM/batlow", "SCM/batlow",
               "SCM/broc",  "gmt/categorical","gmt/categorical", ]

new_index = [0,1,2,3,4,5,6,8,7,9,10,11,12,13,14,15]

#new_index = [4,3,15,6,7,0, 14, 10,16, 8, 9,2, 13, 12, 8, 11, ]

obs = obs.reindex(new_index)

#obs.index = np.arange(0,len(obs))

pd.options.display.width = 370
pd.options.display.max_colwidth = 16




obs_dict = obs.to_dict(orient='records')

obs.set_index(['OBS_REF'], inplace=True)



obs

,OBS_AFR,LABELS_gmt,LABELS,UNITS,UNITS_gmt,V_RANGE,V_RANGE_AFR,CMAPS
OBS_REF,,,,,,,,
CTD,CTD,CTD,CTD,km,km,"(0, 50)","(0, 50)",SCM/bamako
SI,SI,Shape index,Shape index,si,si,"(-1, 1)","(-1, 1)",SCM/broc
LAB,LAB,LAB,LAB,km,km,"(0, 300)","(50, 250)",SCM/bamako
MOHO,MOHO,Moho,Moho,km,km,"(15, 60)","(20, 50)",SCM/bamako
SV,SV_SPEED,S@_v@ 150km,$S_v$ @150km,$\delta$$S_v$ %,km/s,"(-0.075, 0.075)","(-0.075, 0.075)",SCM/roma
PV,PV_SPEED,P@_v@ 150km,$P_v$ @150km,$\delta$$P_v$ %,km/s,"(-0.02, 0.02)","(-0.02, 0.02)",SCM/roma
GEOID,GEOID,Geoid,Geoid,m,m,"(-45, 45)","(-45, 45)",SCM/bamako
DEM,DEM,DEM,DEM,m,m,"(-2200, 2200)","(-2200, 2200)",SCM/oleron
FA,FA,Free air,Free air,mGal,mGal,"(-100, 100)","(-100, 100)",SCM/broc


In [6]:

target = 'GHF'
coord = ['lon', 'lat']



#######

features_ex = []

features = obs.index.tolist()


features_ex = copy.deepcopy(features)
features_ex.extend(coord)
features_ex.append(target)





In [7]:
spacing_world = 0.2 # 12 arcmin


# create coordinates
cols_world =np.linspace(world_lon_min, world_lon_max, 5* (world_lon_max - world_lon_min)-1 )
rows_world = np.linspace(world_lat_min, world_lat_max, 5* (world_lat_max - world_lat_min) -1 )
row_meshgrid_world ,col_meshgrid_world = np.meshgrid( rows_world, cols_world, indexing='ij')


# put data into a dataset
ds_world = xr.Dataset(
    data_vars=dict(
         #variables=(["Y", "X"])
    ),
    coords=dict(
        
        YV=(["Y", "X"], row_meshgrid_world),
        XV=(["Y", "X"], col_meshgrid_world),
        lat=(["Y", "X"], row_meshgrid_world),
        lon=(["Y", "X"], col_meshgrid_world),
        Y=(["Y"], rows_world),
        X=(["X"], cols_world),
        
    ),
    attrs=dict(description="coords with matrices"),
)


#######


spacing_afr = 0.5


# create coordinates
cols_afr =np.linspace(afr_lon_min, afr_lon_max, 2* (afr_lon_max - afr_lon_min)-1 )
rows_afr = np.linspace(afr_lat_min, afr_lat_max, 2* (afr_lat_max - afr_lat_min) -1 )
row_meshgrid_afr ,col_meshgrid_afr = np.meshgrid( rows_afr, cols_afr, indexing='ij')


# put data into a dataset
ds_afr = xr.Dataset(
    data_vars=dict(
         #variables=(["Y", "X"])
    ),
    coords=dict(
        
        YV=(["Y", "X"], row_meshgrid_afr),
        XV=(["Y", "X"], col_meshgrid_afr),
        lat=(["Y", "X"], row_meshgrid_afr),
        lon=(["Y", "X"], col_meshgrid_afr),
        Y=(["Y"], rows_afr),
        X=(["X"], cols_afr),
        
    ),
    attrs=dict(description="coords with matrices"),
)



# Global datasets

In [8]:
# zipped files to be downaloded and extracted automatically in their respective directories 
ctd_url = 'https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/bvz2jz99xh-2.zip'
emag2v3_url =  'https://www.ngdc.noaa.gov/geomag/data/EMAG2/EMAG2_V3_20170530/EMAG2_V3_20170530_UpCont.tif'
etopo_url =  'https://www.ngdc.noaa.gov/mgg/global/relief/ETOPO1/data/bedrock/grid_registered/georeferenced_tiff/ETOPO1_Bed_g_geotiff.zip'
litho_rho_url = 'https://b8ea67cb-f91c-4c4e-ba10-1f57eef81dff.filesusr.com/archives/7f2185_e894260d28d94e1ea1bdfcc04d213ef0.zip?dn=LithoRef_model.zip'
glim_url = 'https://doi.pangaea.de/10013/epic.39939.d001'


In [9]:


# correct values to limit them [-1,1]
def mag_log(data, C=400, clip_min=-1, clip_max = 1):
     return np.clip(np.sign(data)*np.log(1+np.abs(data)/C), clip_min, clip_max)


emag2v3_path = pooch.retrieve(
    url=emag2v3_url,
    known_hash=None,
    progressbar=True,
)



#The emag dataset needs to be adjusted from 0 - 360 to -180 180

kwargs = {'dstSRS':'EPSG:4326', 
          'srcSRS':'EPSG:4326',
          'format' : 'GTiff', 
          'outputBounds': (0, -90, 360, 90) }


print('Importing.....', end='\n\n')
emag2v3_dir = DIR/ 'Dataset'/ 'References'/'g_Magnetic'
emag2v3_f = 'EMAG2_V3_20170530_UpCont_m0.tif'
gdal.Warp(emag2v3_f, emag2v3_path,  **kwargs)


try:
    #Path(DIR/emag2v3_f).rename(emag2v3_dir /emag2v3_f)
    shutil.move(DIR/emag2v3_f, emag2v3_dir /emag2v3_f)
except OSError as e:
    print(f"An error has occurred. Continuing anyways: {e}", end='\n')

      
print('Interpolating.....', end='\n\n')

for i in tqdm_notebook(range(1), desc = 'Processing: '):
    sleep(0.01)
    observable = 'EMAG2'
    ds_world[observable] = (('Y', 'X'),  
                         utils.read_raster(f_name = emag2v3_dir /emag2v3_f, 
                                     ds=ds_world,
                                   crs_from=crs_from, crs_to=crs_to, no_data = -99999))
    
    observable = 'EMAG2_CLASS'
    ds_world[observable] = (('Y', 'X'),  mag_log(ds_world['EMAG2'].values)) 


Importing.....

Interpolating.....



Processing:   0%|          | 0/1 [00:00<?, ?it/s]

Raster bounds: BoundingBox(left=0.0, bottom=-90.0, right=360.0, top=90.0) (5400, 10800)


In [10]:


glim_path = pooch.retrieve(
    url=glim_url,
    known_hash=None,
    progressbar=True,
)


print('Unzipping.....', end='\n\n')

archive = ZipFile(glim_path, 'r', )



glim_dir = DIR/ 'Dataset'/ 'References'/'p_Global_Lithological_Map'
glim_f = archive.extract(archive.filelist[1], path= glim_dir)

print('Interpolating.....', end='\n\n')

for i in tqdm_notebook(range(1), desc = 'Processing: '):
    sleep(0.01)
    observable = 'GLIM'
    ds_world[observable] = (('Y', 'X'), np.around(utils.read_raster(
    f_name=glim_f, skiprows=6,  ds =ds_world,
        interpol='nearest', crs_from=crs_from, crs_to=crs_to),0))


    ds_world[observable] = ds_world[observable].where(ds_world[observable].data>0, 16)
    ds_world[observable] = ds_world[observable].where(ds_world[observable].data<16,16)


glim_classes = ["su", "vb" ,"ss", "pb" , "sm" ,"sc" ,"va", "mt" ,"pa",
"vi", "wb" ,"py", "pi" ,"ev" ,"nd" ,"ig"]

Unzipping.....

Interpolating.....



Processing:   0%|          | 0/1 [00:00<?, ?it/s]

Raster bounds: BoundingBox(left=-180.0, bottom=-90.0, right=180.0, top=90.0) (360, 720)


In [11]:


litho_rho_path = pooch.retrieve(
    url=litho_rho_url,
    known_hash=None,
    progressbar=True,
)


print('Unzipping.....', end='\n\n')
archive = ZipFile(litho_rho_path, 'r', )


litho_rho_dir = DIR/ 'Dataset'/ 'References'/'i_Lithosphere_Average_Density'
lith_roh_f = archive.extract(archive.filelist[0], path=litho_rho_dir)

lith_df = pd.read_csv(lith_roh_f ,header=None,  engine='python',skiprows=9, 
                      encoding="ISO-8859-1", delim_whitespace=True)
print(f'min lon : {lith_df.iloc[:,0].min()}')
print(f'min lat : {lith_df.iloc[:,1].min()}')
print(f'min rho_c: {lith_df.iloc[:,5].min()}')

print(f'max lon : {lith_df.iloc[:,0].max()}')
print(f'max lat : {lith_df.iloc[:,1].max()}')
print(f'max rho_c: {lith_df.iloc[:,5].max()}')


litho_c_df = lith_df.iloc[:,[0,1,5]]

litho_c_df.columns = [0,1,2]

litho_c_df[2] = litho_c_df[2].round(4)

###


print('#################')

crust_rho_dir = DIR/ 'Dataset'/ 'References'/'i_Crustal_Average_Density'
archive.extract(archive.filelist[0], path=crust_rho_dir)

print(f'min lon : {lith_df.iloc[:,0].min()}')
print(f'min lat : {lith_df.iloc[:,1].min()}')
print(f'min rho_l: {lith_df.iloc[:,6].min()}')

print(f'max lon : {lith_df.iloc[:,0].max()}')
print(f'max lat : {lith_df.iloc[:,1].max()}')
print(f'max rho_l: {lith_df.iloc[:,6].max()}')

litho_l_df = lith_df.iloc[:,[0,1,6]]
litho_l_df.columns = [0,1,2]

litho_l_df[2] = litho_l_df[2].round(4)

print('\nInterpolating.....', end='\n\n')

for observable, df in tqdm_notebook(
    zip(['RHO_L', 'RHO_C'], [litho_l_df, litho_c_df]), 
    total = 2, desc = 'Processing: '):
    sleep(0.01)

    ds_world[observable] =  (('Y', 'X'), utils.read_numpy(
        data=df.values, ds=ds_world, crs_from=crs_from, 
        crs_to=crs_to, interpol='IDW'))




100%|###############################################| 685k/685k [00:00<?, ?B/s]
SHA256 hash of downloaded file: 0d63cb974d8a0d15387de1977c42f3a6d1bff9049235d7453172c6a3f94b6626
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.


Unzipping.....

min lon : -179.0
min lat : -89.0
min rho_c: 2558.31
max lon : 179.0
max lat : 89.0
max rho_c: 2999.94
#################
min lon : -179.0
min lat : -89.0
min rho_l: 3234.79
max lon : 179.0
max lat : 89.0
max rho_l: 3392.54

Interpolating.....



Processing:   0%|          | 0/2 [00:00<?, ?it/s]

In [12]:

ctd_path = pooch.retrieve(
    url=ctd_url,
    known_hash=None,
    progressbar=True,
)

print('\nUnzipping.....', end='\n\n')
archive = ZipFile(ctd_path, 'r', )
#glim_f = archive.read('glim_wgs84_0point5deg.txt.asc')

ctd_dir = DIR/ 'Dataset'/ 'References'/'f_CTD'
ctd_f = archive.extract(archive.filelist[0], path=ctd_dir)
CTD_df = pd.read_csv(ctd_f ,header=None,  delim_whitespace=True)

print(f'min lon : {CTD_df.iloc[:,0].min()}')
print(f'min lat : {CTD_df.iloc[:,1].min()}')
print(f'min z: {CTD_df.iloc[:,2].min()}')

print(f'max lon : {CTD_df.iloc[:,0].max()}')
print(f'max lat : {CTD_df.iloc[:,1].max()}')
print(f'max z: {CTD_df.iloc[:,2].max()}')

CTD_df[2] = CTD_df[2].round(4)

print('\nInterpolating.....', end='\n\n')

for i in tqdm_notebook(range(1), desc = 'Processing: '):
    sleep(0.01)
    observable = 'CTD'
    ds_world[observable] =  (('Y', 'X'), utils.read_numpy(
        data=CTD_df.values, ds=ds_world, crs_from=crs_from, 
        crs_to=crs_to, interpol='IDW'))



Unzipping.....

min lon : -179.5
min lat : -88.5
min z: -7.4117378
max lon : 179.5
max lat : 83.5
max z: 160.59855

Interpolating.....



Processing:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
etopo_path = pooch.retrieve(
    url=etopo_url,
    known_hash=None,
    progressbar=True,
)

print('Unzipping.....', end='\n\n')
archive = ZipFile(etopo_path, 'r', )


dem_dir = DIR/ 'Dataset'/ 'References'/'h_Elevation'
dem_f = archive.extract(archive.filelist[0], path=dem_dir)

print('\nInterpolating.....', end='\n\n')

for i in tqdm_notebook(range(1), desc = 'Processing: '):
    sleep(0.01)

    observable = 'DEM'
    ds_world[observable] = (('Y', 'X'), 
                            utils.read_raster(dem_f,   ds = ds_world, crs_from=crs_from, crs_to=crs_to))

Unzipping.....


Interpolating.....



Processing:   0%|          | 0/1 [00:00<?, ?it/s]

Raster bounds: BoundingBox(left=-180.00833333333333, bottom=-90.00833333333328, right=180.00833333333333, top=90.00833333333334) (10801, 21601)


In [14]:
# automatic downloads
moho_url = 'https://zenodo.org/record/5730195/files/Global_Moho_WINTERC-G.xyz?download=1'
lab_url = 'https://zenodo.org/record/5771863/files/WINTERC-G_LAB.lis?download=1'




In [15]:
lab_path = pooch.retrieve(
    url=lab_url,
    known_hash=None,
    progressbar=True,
)

print('Importing.....', end='\n\n')

lab_df = pd.read_csv(lab_path ,header=None,  engine='python',skiprows=1,  encoding="ISO-8859-1", delim_whitespace=True)

lab_df.set_index([0], inplace=True)


print(f'min lon : {lab_df.iloc[:,0].min()}')
print(f'min lat : {lab_df.iloc[:,1].min()}')
print(f'min lab: {lab_df.iloc[:,2].min()}')

print(f'max lon : {lab_df.iloc[:,0].max()}')
print(f'max lat : {lab_df.iloc[:,1].max()}')
print(f'max lab : {lab_df.iloc[:,2].max()}')

lab_df[2]  = lab_df[2].round(4)

print('\nInterpolating.....', end='\n\n')

for i in tqdm_notebook(range(1), desc = 'Processing: '):
    sleep(0.01)
    observable = 'LAB'
    ds_world[observable] =  (('Y', 'X'), utils.read_numpy(
        data=lab_df.values, ds=ds_world, crs_from=crs_from,
        crs_to=crs_to, set_center=True, interpol='IDW'))


Importing.....

min lon : 0.0
min lat : -88.2
min lab: 45.0
max lon : 359.0
max lat : 88.2
max lab : 300.0

Interpolating.....



Processing:   0%|          | 0/1 [00:00<?, ?it/s]

In [16]:
moho_path = pooch.retrieve(
    url=moho_url,
    known_hash=None,
    progressbar=True,
)

print('Importing.....', end='\n\n')

moho_df = pd.read_csv(moho_path ,header=None,  engine='python',skiprows=1,  encoding="ISO-8859-1", delim_whitespace=True)




print(f'min lon : {moho_df.iloc[:,0].min()}')
print(f'min lat : {moho_df.iloc[:,1].min()}')
print(f'min si: {moho_df.iloc[:,2].min()}')

print(f'max lon : {moho_df.iloc[:,0].max()}')
print(f'max lat : {moho_df.iloc[:,1].max()}')
print(f'max si : {moho_df.iloc[:,2].max()}')

moho_df[2]  = moho_df[2].round(4)

print('\nInterpolating.....', end='\n\n')

for i in tqdm_notebook(range(1), desc = 'Processing: '):
    sleep(0.01)
    observable = 'MOHO'
    ds_world[observable] =  (('Y', 'X'), utils.read_numpy(
        data=moho_df.values, ds=ds_world, crs_from=crs_from,
        crs_to=crs_to, interpol='IDW',
        set_center=True, ))


Importing.....

min lon : 0.0
min lat : -89.75
min si: 8.33079
max lon : 359.5
max lat : 89.75
max si : 68.6781

Interpolating.....



Processing:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
#Datasets to be downloaded directly from their respective sources
# then to be placed in their respective directories
# sources of datasets
# pv wave : https://www.earth.ox.ac.uk/~smachine/cgi/index.php?page=tomo_depth
# Sv wave : https://www.dropbox.com/s/te4inov1586vus4/SL2013sv_0.5d-grd_v2.1.tar.bz2?dl=0
# Geoid, Free-air, Bouguer : http://icgem.gfz-potsdam.de/calcgrid
# shape index : https://www.3dearth.uni-kiel.de/en/public-data-products/copy_of_depth-to-moho-boundary
# Tectonic regions : https://schaeffer.ca/models/sl2013sv-tectonic-regionalization/
# Holocenes volcanoes https://volcano.si.edu/volcanolist_holocene.cfm
# Pleistocene volcanoes https://volcano.si.edu/volcanolist_pleistocene.cfm
#NGHF : 'https://agupubs.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1029%2F2019GC008389&file=2019GC008389-sup-0004-Data_Set_SI-S02.zip'




pv_g_f = DIR / 'Dataset'/'References'/'e_PV'/'DETOX-P1.txt'
sv_g_f = DIR / 'Dataset'/'References'/'d_SV'/'SL2013sv_dVs_25km-0.5d_50+.xyz'
geoid_f = DIR / 'Dataset'/ 'References'/'l_Geoid' / 'EIGEN-6C4_geoid.gdf'
fa_f = DIR / 'Dataset'/ 'References'/'k_Free_Air' / 'EIGEN-6C4_gravity_anomaly_cl.gdf'
bg_f = DIR / 'Dataset'/ 'References'/'m_Bouguer' / 'EIGEN-6C4_gravity_anomaly_bg.gdf'
si_f = DIR / 'Dataset'/ 'References'/'n_Shape_Index' / 'GOCE_Curvature_Topo_Iso_Corr.txt'
reg_f = DIR / 'Dataset'/'References'/'o_Tectonic_Regionalization'/'SL2013sv_Cluster_2d'
volcs_hol_f =   DIR / 'Dataset'/ 'References'/'c_Distance_to_nearest_volcano'/'GVP_Volcano_List_Holocene.xlsx'
volcs_plei_f =  DIR / 'Dataset'/ 'References'/'c_Distance_to_nearest_volcano'/'GVP_Volcano_List_Pleistocene.xlsx'
hq_f = DIR / 'Dataset'/ 'References'/ 'q_Heat_Flow'/'NGHF.csv'




In [19]:

reg_df = pd.read_csv(reg_f ,header=None,   delim_whitespace=True)
print(f'min lon : {reg_df.iloc[:,0].min()}')
print(f'min lat : {reg_df.iloc[:,1].min()}')
print(f'min si: {reg_df.iloc[:,2].min()}')

print(f'max lon : {reg_df.iloc[:,0].max()}')
print(f'max lat : {reg_df.iloc[:,1].max()}')
print(f'max si : {reg_df.iloc[:,2].max()}')


print('\nInterpolating.....', end='\n\n')

for i in tqdm_notebook(range(1), desc = 'Processing: '):
    sleep(0.01)
    observable = 'REG'
    ds_world[observable] =  (('Y', 'X'), utils.read_numpy(
        data=reg_df.values, ds=ds_world, crs_from=crs_from,
        crs_to=crs_to, interpol='nearest'))


min lon : -180.0
min lat : -90.0
min si: 1
max lon : 180.0
max lat : 90.0
max si : 6

Interpolating.....



Processing:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:

print('Importing.....')
si_df = pd.read_csv(si_f ,header=None,  engine='python',skiprows=1, 
                    encoding="ISO-8859-1", delim_whitespace=True)



print(f'min lon : {si_df.iloc[:,0].min()}')
print(f'min lat : {si_df.iloc[:,1].min()}')
print(f'min si: {si_df.iloc[:,2].min()}')

print(f'max lon : {si_df.iloc[:,0].max()}')
print(f'max lat : {si_df.iloc[:,1].max()}')
print(f'max si : {si_df.iloc[:,2].max()}')

si_f_df = si_df[[0,1,2]]
si_f_df[2]  = si_f_df[2].round(4)

print('\nInterpolating.....', end='\n\n')

for i in tqdm_notebook(range(1), desc = 'Processing: '):
    sleep(0.01)
    observable = 'SI'
    ds_world[observable] =  (('Y', 'X'), utils.read_numpy(
        data=si_f_df.values, ds=ds_world, crs_from=crs_from,
        crs_to=crs_to, interpol='IDW'))

Importing.....
min lon : -180.0
min lat : -89.8
min si: -0.99939
max lon : 949.0
max lat : 89.8
max si : 0.99932

Interpolating.....



Processing:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:

print('Importing.....', end='\n\n')

geoid_df = pd.read_csv(geoid_f ,header=None,  engine='python',skiprows=36,  encoding="ISO-8859-1", delim_whitespace=True)

print(f'min lon : {geoid_df.iloc[:,0].min()}')
print(f'min lat : {geoid_df.iloc[:,1].min()}')
print(f'min geoid: {geoid_df.iloc[:,2].min()}')

print(f'max lon : {geoid_df.iloc[:,0].max()}')
print(f'max lat : {geoid_df.iloc[:,1].max()}')
print(f'max z: {geoid_df.iloc[:,2].max()}')


geoid_df = geoid_df.round(4)


#####

print('###############')
fa_df = pd.read_csv(fa_f ,header=None,  engine='python',skiprows=36,  encoding="ISO-8859-1", delim_whitespace=True)


print(f'min lon : {fa_df.iloc[:,0].min()}')
print(f'min lat : {fa_df.iloc[:,1].min()}')
print(f'min geoid: {fa_df.iloc[:,2].min()}')

print(f'max lon : {fa_df.iloc[:,0].max()}')
print(f'max lat : {fa_df.iloc[:,1].max()}')
print(f'max z: {fa_df.iloc[:,2].max()}')

fa_df = fa_df.round(4)


#####

print('###############')
bg_df = pd.read_csv(bg_f ,header=None,  engine='python',skiprows=37,  encoding="ISO-8859-1", delim_whitespace=True)


print(f'min lon : {bg_df.iloc[:,0].min()}')
print(f'min lat : {bg_df.iloc[:,1].min()}')
print(f'min geoid: {bg_df.iloc[:,2].min()}')

print(f'max lon : {bg_df.iloc[:,0].max()}')
print(f'max lat : {bg_df.iloc[:,1].max()}')
print(f'max z: {bg_df.iloc[:,2].max()}')

bg_df = bg_df.round(4)


print('\nInterpolating.....', end='\n\n')

for observable, df in tqdm_notebook(zip(['GEOID', 'FA' , 'BG'], [geoid_df, fa_df, bg_df]), 
                                    total=3, desc = 'Processing: '):
    sleep(0.01)
    ds_world[observable] =  (('Y', 'X'), utils.read_numpy(
        data=df.values, ds=ds_world, crs_from=crs_from,
        crs_to=crs_to, interpol='IDW'))


Importing.....

min lon : -180.0
min lat : -90.0
min geoid: -106.184427114513
max lon : 180.0
max lat : 90.0
max z: 84.917290827278
###############
min lon : -180.0
min lat : -90.0
min geoid: -301.300980351904
max lon : 180.0
max lat : 90.0
max z: 522.25583386214
###############
min lon : -180.0
min lat : -90.0
min geoid: -631.218235051554
max lon : 180.0
max lat : 90.0
max z: 455.456168986875

Interpolating.....



Processing:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:

print('Importing.....', end='\n\n')

pv_g_df = pd.read_csv(pv_g_f, skiprows=3,
                              header=None, engine='python', skip_blank_lines=True,   na_filter=True, delim_whitespace=True)


print(f'lon min : {pv_g_df.iloc[:,0].min()}')
print(f'lat min : {pv_g_df.iloc[:,1].min()}')
print(f'lon max : {pv_g_df.iloc[:,0].max()}')
print(f'lat max : {pv_g_df.iloc[:,1].max()}')
print(f'pv min before: {pv_g_df[2] .min()}')
print(f'pv max before: {pv_g_df[2] .max()}')


# remove % by * 0.01 'dVp(%)'

pv_g_df[2] = pv_g_df.iloc[:,2].multiply(0.01).round(4)
print(f'pv min after: {pv_g_df[2] .min()}')
print(f'pv max after: {pv_g_df[2] .max()}')


print('#####')

print('\nInterpolating.....', end='\n\n')

for i in tqdm_notebook(range(1), desc = 'Processing: '):
    sleep(0.01)
    observable = 'PV'
    ds_world[observable] =  (('Y', 'X'), utils.read_numpy(
        data=pv_g_df.values, ds=ds_world, crs_from=crs_from,
        set_center=True, crs_to=crs_to, interpol='IDW'))

In [ ]:

print('Importing.....', end='\n\n')

sv_init_df = pd.read_csv(sv_g_f,header=None, skiprows=1, skip_blank_lines=True, 
                      na_filter=True, delim_whitespace=True)

print(f'lon min : {sv_init_df.iloc[:,1].min()}')
print(f'lat min : {sv_init_df.iloc[:,2].min()}')
print(f'lon max : {sv_init_df.iloc[:,1].max()}')
print(f'lat max : {sv_init_df.iloc[:,2].max()}')
print(f'sv min before: {sv_init_df[5] .min()}')
print(f'sv max before: {sv_init_df[5] .max()}')
# remove % by * 0.01 'dVs(%)'

sv_init_df[5] = sv_init_df.iloc[:,5].multiply(0.01).round(4)
print(f'sv min after: {sv_init_df[5] .min()}')
print(f'sv max after: {sv_init_df[5] .max()}')

print('#####')

sv_g_150_df = sv_init_df[sv_init_df[0]==150][[1,2,5]]
#seismic_w_sv_df = copy.deepcopy(seismic_w_sv_df_150_df.iloc[:,[1,2,5]]


sv_g_150_df.columns = [0,1,2]



print('\nInterpolating.....', end='\n\n')

for i in tqdm_notebook(range(1), desc = 'Processing: '):
    sleep(0.01)
    observable = 'SV'
    ds_world[observable] =  (('Y', 'X'), utils.read_numpy(
        data=sv_g_150_df.values, ds=ds_world, crs_from=crs_from,
        crs_to=crs_to, interpol='IDW'))   

In [ ]:
volc_max_dist = 100
a_max_dist = 250


volcs_hol_df = pd.read_excel(volcs_hol_f, header=1)
volcs_plei_df = pd.read_excel(volcs_plei_f, header=1)
volc_df = pd.concat([volcs_hol_df, volcs_plei_df])

print(volc_df[['Latitude','Longitude']])
cols = volc_df.columns.to_list()
cols[1], cols[0] = cols[0], cols[1]
volc_df.loc[:,cols]

print(f' Hol: {len(volcs_hol_df)}')
print(f' Plei: {len(volcs_plei_df)}')
print(f' Total: {len(volc_df)}')

volc_df = volc_df[['Latitude','Longitude']]
volc_df.columns = [ 'lon', 'lat']


volc_df = volc_df.round(2)

print(f" lon: {volc_df['lon'].min()} {volc_df['lon'].max()}")
print(f" lat: {volc_df['lat'].min()} {volc_df['lat'].max()}")

In [ ]:
# visual inspection of imported datasets
observables = ['RHO_L', 'RHO_C', 'CTD', 'DEM', 'GLIM', 'EMAG2_CLASS', 'MOHO', 'LAB',
              'SV', 'PV', 'REG', 'BG', 'FA', 'GEOID', 'SI']

data_types = [True,True, True, True, False , True , True,True, 
              True,True, False, True,True, True,True, ]

scales = [False,False,False,False, False, True, False,False,          
          False,False, False, False, False, False, False,]



for observable, data_type, scale in tqdm_notebook(zip(observables, data_types, scales), 
                                          total=15, desc = 'Processing: '):
    print('Plotting.....', end='\n\n')
    sleep(0.01)
#for observable, data_type, scale in zip(observables, data_types, scales):
    label = obs.loc[observable, 'LABELS_gmt']
    unit = obs.loc[observable, 'UNITS_gmt']
    if (observable == 'GLIM') or (observable == 'REG') :
        series=(obs.loc[observable, 'V_RANGE'][0], obs.loc[observable, 'V_RANGE'][1]+1, 1)
    else:
        min_range = obs.loc[observable, 'V_RANGE'][0]
        max_range = obs.loc[observable, 'V_RANGE'][1]
        series=(min_range, max_range,(max_range - min_range)/100 )
    
    fig = pygmt.Figure()
    pygmt.makecpt(
        cmap=obs.loc[observable, 'CMAPS'], #temp 19lev

        #cmap='lajolla',
        #series= "1/17/1",
        #truncate = '-2000/2000',
       # categorical=list(range(1,17)),
        continuous=data_type,
        reverse=scale,
        
       series = series,
        # convert ['Adelie', 'Chinstrap', 'Gentoo'] to 'Adelie,Chinstrap,Gentoo'
        #color_model="+c" + ",".join(glim_classes),
    )



    fig.basemap(frame=True, region=region_world, projection=projection,)
    #avoid interpolation
    fig.grdimage(ds_world[observable],
                 interpolation='n+a' 
                )
    



    fig.coast(
        #region = 'JP',
        #shorelines=0.5,
        #water="lightblue", 
        shorelines="0.1p,black",
        #borders=["1/0.001p,black"],
        lakes="lightblue",
        rivers="lightblue" ,
        #borders=["1/0.5p,black"],
        #water='white',
        )
    fig.colorbar(frame=["af", f'x+l"{label}"\t\t{unit}'])
    fig.show(width=1000)
    #fig.savefig("north-america-pygmt-map.png",crop=True, dpi=300, transparent=True)


# Africa

In [ ]:
# we start with shared observables with global models


for i in tqdm_notebook(range(1), desc = 'Processing: '):
    sleep(0.01)

    ds_afr['EMAG2'] = (('Y', 'X'),  utils.read_raster(f_name = emag2v3_dir /emag2v3_f, 
                                            ds=ds_afr, crs_to=crs_to, crs_from=crs_from, 
                                            interpol='nearest', no_data = -99999))
    ds_afr['EMAG2_CLASS'] = (('Y', 'X'),  mag_log(ds_afr['EMAG2'].values)) 


for i in tqdm_notebook(range(1), desc = 'Processing: '):
    sleep(0.01)
    observable = 'DEM'
    ds_afr[observable] = (('Y', 'X'), utils.read_raster(
            f_name=dem_f, ds=ds_afr, interpol='nearest', crs_from=crs_from, crs_to=crs_to))





####

for i in tqdm_notebook(range(1), desc = 'Processing: '):
    sleep(0.01)
    observable = 'GLIM'
    ds_afr[observable] = (('Y', 'X'), np.around(utils.read_raster(
    f_name=glim_f, skiprows=6, interpol='nearest', ds=ds_afr, crs_from=crs_from, crs_to=crs_to),0))


    ds_afr[observable] = ds_afr[observable].where(ds_afr[observable].data>0, 16)
    ds_afr[observable] = ds_afr[observable].where(ds_afr[observable].data<16,16)




###




observables = ['BG', 'FA', 'GEOID', 'SI', 'REG', 'MOHO', 'LAB', 'CTD', 'RHO_L', 'RHO_C' ]
dfs = [bg_df, fa_df, geoid_df, si_df, reg_df, moho_df, lab_df, CTD_df, litho_l_df, litho_c_df]
interpols = ['IDW', 'IDW', 'IDW', 'IDW', 'nearest', 'IDW', 'IDW', 'IDW', 'IDW', 'IDW']
centerings = [False, False, False, False, False, True, True, False, False, False]





for observable, df, interpol, centering in tqdm_notebook(
    zip(observables, dfs, interpols, centerings), 
                                          total=10, desc = 'Processing: '):

    sleep(0.01)
    ds_afr[observable] =  (('Y', 'X'), utils.read_numpy(
    data=df.values, ds=ds_afr, crs_from=crs_from, crs_to=crs_to, interpol=interpol,
    set_center=centering))

print('Terminated')


In [ ]:
# get the position of volcanoes



geod = proj.Geod(ellps='WGS84' )
vol_int = volc_df[volc_df['lon'].between(afr_lon_min, afr_lon_max) 
                     & volc_df['lat'].between(afr_lat_min, afr_lat_max)]
v_lon = vol_int['lon'].values
v_lat = vol_int['lat'].values

v_n = len(v_lon)

v_k = 1

    
ny = len(ds_afr.Y)
nx = len(ds_afr.X)
nn = (ny, nx)
volcs = np.zeros(nn)

lats = ds_afr['lat'].values
lons = ds_afr['lon'].values

# this process is slow to map all grid

for x in tqdm_notebook(range(nx), desc='lon: ' ):
    for y in tqdm_notebook(range(ny), desc='lat: ', leave=False ):
        sleep(0.01)
        lat = lats[y,x]
        lon = lons[y,x]
        _, _, ds_ = geod.inv(v_n*[lon], v_n*[lat], v_lon, v_lat)
        idx = np.argpartition(ds_, v_k)[:v_k]
        volcs[y,x] = np.sum(np.take(ds_, idx))/v_k/km

ds_afr['VOLC_DIST'] = (('Y', 'X'), volcs)
ds_afr['VOLC_DIST_W'] = (('Y', 'X'), np.clip(1 - volcs/volc_max_dist, 0, 1)) 




In [ ]:
# downlaod the modle from https://nlscelli.wixsite.com/ncseismology/af2019

sv_a_f = DIR / 'Dataset'/'References'/'d_SV'/'AF2019_vsvp.mod'

pv_afr_url = 'http://ds.iris.edu/files/products/emc/emc-files/AFRP20-RF-CR1-MOD-3D.r0.0.nc'




In [ ]:
pv_afr_path = pooch.retrieve(
    url=pv_afr_url,
    known_hash=None,
    progressbar=True,
)

depth_150_km = 2

print('Importing.....', end='\n\n')

### dVp
dvp = xr.load_dataset(pv_afr_path)['dvp'][depth_150_km,:,:].values.reshape(-1,1)
dvp = np.round(dvp * 0.01, 4)
pv_lon = xr.load_dataset(pv_afr_path)['longitude'].values
pv_lat = xr.load_dataset(pv_afr_path)['latitude'].values
lat_mesh ,lon_mesh = np.meshgrid( pv_lat, pv_lon, indexing='ij')

lons = lon_mesh.reshape(-1,1)
lats = lat_mesh.reshape(-1,1)
dvp_arr = np.append(lons, lats, axis=1)
dvp_arr = np.append(dvp_arr, dvp, axis=1)



### VP velocity


pv = xr.load_dataset(pv_afr_path)['vp'][depth_150_km,:,:].values.reshape(-1,1)

vp = np.round(pv, 4)
vp_arr = np.append(lons, lats, axis=1)
vp_arr = np.append(vp_arr, vp, axis=1)


####

print('\nInterpolating.....', end='\n\n')


for observable, arr in tqdm_notebook(
    zip(['PV', 'PV_Velocity' ], [dvp_arr, vp_arr]), 
                                          total=2, desc = 'Processing: '):

    ds_afr[observable] =  (('Y', 'X'), utils.read_numpy(
        data=arr, ds=ds_afr, crs_from=crs_from, crs_to=crs_to, interpol='IDW'))



In [ ]:

print('Importing.....', end='\n\n')

sv_init_df = pd.read_csv(sv_a_f   , skip_blank_lines=True, 
                      na_filter=True, delim_whitespace=True)


print(f'lon min : {sv_init_df["#lon"].min()}')
print(f'lat min : {sv_init_df["lat"].min()}')
print(f'lon max : {sv_init_df["#lon"].max()}')
print(f'lat max : {sv_init_df["lat"].max()}')
print(f'dVs min before: {sv_init_df["dVs"] .min()}')
print(f'dVs max before: {sv_init_df["dVs"] .max()}')

#calculate percentile dvs

sv_init_df["dVs%"] = sv_init_df["dVs"] / sv_init_df["Vs_ref"]
print(f'dVs % min after: {sv_init_df["dVs%"].min()}')
print(f'dVs % max after: {sv_init_df["dVs%"].max()}')


dvs_df = sv_init_df[sv_init_df['depth']==150][['#lon','lat' , "dVs%"]]

dvs_df["dVs%"]  = dvs_df["dVs%"].round(4)
    
dvs_df.columns = [0,1,2]

#######
# SV africa speed


vs_df = sv_init_df[sv_init_df['depth']==150][['#lon','lat' , 'Vs_abs']]


    
vs_df['Vs_abs'] = vs_df['Vs_abs'].round(4)/1000   


vs_df.columns = [0,1,2]


print('\nInterpolating.....', end='\n\n')

for observable, df in tqdm_notebook(
    zip(['SV', 'SV_Velocity' ], [dvs_df, vs_df]), 
                                          total=2, desc = 'Processing: '):

    ds_afr[observable] =  (('Y', 'X'), utils.read_numpy(
        data=df.values, ds=ds_afr, crs_from=crs_from, crs_to=crs_to, interpol='IDW'))



In [ ]:
# visual inspection of imported datasets


data_types = [True,True, True, True, True , True , True,True, 
              True, True,True, True,True, True, False, False ]
scales = [False,False,False,False, False, False, False,False,
          
          False,False, False, False, False, False, False, False]

for observable, data_type, scale in tqdm_notebook(zip(obs.index, data_types, scales), 
                                          total=16, desc = 'Processing: '):
    print('Plotting.....', end='\n\n')
    sleep(0.01)
    label = obs.loc[observable, 'LABELS_gmt']
    unit = obs.loc[observable, 'UNITS_gmt']
    if (observable == 'GLIM') or (observable == 'REG') :
        series=(obs.loc[observable, 'V_RANGE'][0], obs.loc[observable, 'V_RANGE'][1]+1, 1)
    else:
        min_range = obs.loc[observable, 'V_RANGE'][0]
        max_range = obs.loc[observable, 'V_RANGE'][1]
        series=(min_range, max_range,(max_range - min_range)/100 )

    fig = pygmt.Figure()
    cmap = pygmt.makecpt(
        cmap=obs.loc[observable, 'CMAPS'], #temp 19lev
        #cmap='lajolla',
        #series= "1/17/1",
        #truncate = '-2000/2000',
       # categorical=list(range(1,17)),
        continuous=data_type,
        reverse=scale,

       series = series,
        # convert ['Adelie', 'Chinstrap', 'Gentoo'] to 'Adelie,Chinstrap,Gentoo'
        #color_model="+c" + ",".join(glim_classes),
    )



    fig.basemap(frame=True, region=region_afr, projection=projection,)
    #avoid interpolation
    if (observable == 'GLIM') or (observable == 'REG') :
        fig.grdimage(ds_afr[observable],
                     interpolation='n+a' 
                    )
    else:

        fig.grdimage(ds_afr[observable]
                    )
    fig.coast(
        #shorelines=0.5,
        #water="lightblue", 
        shorelines="0.1p,black",
        #borders=["1/0.001p,black"],
        lakes="lightblue",
        rivers="lightblue" ,
        #dcw="=AS,=EU+gdarkgrey",
        resolution = 'i',
        )
    fig.colorbar(frame=["af", f'x+l"{label}"\t\t{unit}'])
    fig.show(width=600)


function to make inverse distance weight interpolation

# read in dataseys from xyz folder

Interpolation for wolrd

interpolatation for Africa

# 2 - Merge data

In [ ]:
#extract hq ratings

# extracted from agrid Tobias Stal
rabcd = ['A', 'B', 'C', 'D']
rabc = ['A', 'B', 'C']
rab = ['A', 'B']
ra = ['A']
exclude = ['Indian Ocean', 
           'Arctic Ocean', 
           'North Atlantic Ocean', 
           'South Atlantic Ocean', 
           'South Pacific Ocean', 
           'North Pacific Ocean']



hq_in = pd.read_csv(hq_f)
# convert hear to mili


elev_range = (-5000,5000)
elev_cut = -1000
elev_high_cut = -250

fig, ax = plt.subplots(1,2, figsize=(16, 5))
plt.style.use('seaborn-colorblind')


hq_in = hq_in.rename(columns={'heat-flow (mW/m2)' :target, 'longitude': 'lon', 'latitude': 'lat'})
    
ax[1].set_xlim(elev_range)


utils.print_latex_tab_line(d = hq_in[target], c = 'All records')

utils.hq_hist(hq_in, target, ax=ax[0], label = f'All records (N={len(hq_in):,})')


# Remove records with missing position or heat flow value in long lat heat
hq_clean = hq_in.dropna(subset = ['lon', 'lat', target])
utils.print_latex_tab_line(d = hq_clean[target], c = 'Excluded incomplete records')
utils.hq_hist(hq_clean, target,ax=ax[0], label = f'Excluded incomplete records (N={len(hq_clean):,})')

# Remove measurements at low elevation
hq_elev = hq_clean[hq_clean['elevation (m)']>elev_cut]



hq_clean['elevation (m)'].hist(bins=151, ax=ax[1], 
                               range = elev_range, 
                               label = 'Excluded',
                              color = 'brown')

hq_elev['elevation (m)'].hist(bins=151, ax=ax[1], 
                              range = elev_range, 
                              label = 'Used records',
                             color = 'green')


ax[1].axvline(x=elev_cut, color='gray', label=f'Cut-off elevation {elev_cut} m', lw=1)

utils.print_latex_tab_line(d = hq_elev[target], c = f'Excluded deeper than {abs(elev_cut):,} m bsl')
utils.hq_hist(hq_elev, target,ax=ax[0],  label = f'Excluded deeper than {abs(elev_cut)} m (N={len(hq_elev):,})') 

#hq_land = hq_clean.loc[~hq_clean['country'].isin(exclude)] # Exclude oceans 

# Remove any heat flow measurements from high lats for poles
hq_no_pole = hq_elev[hq_elev['lat'].between(world_lat_min, world_lat_max, inclusive='both')]
utils.print_latex_tab_line(d = hq_no_pole[target], c = 'Excluded high lats')
utils.hq_hist(hq_no_pole, target, ax=ax[0], label = f'Excluded high lats (N={len(hq_no_pole):,})')


# Select rabd rathings
hq_rabcd = hq_no_pole.loc[hq_no_pole['code6'].isin(rabcd)] # Only rab measurements
utils.hq_hist(hq_rabcd, target, ax=ax[0], label = f'Rating A-D (N={len(hq_rabcd):,})')
utils.print_latex_tab_line(d = hq_rabcd[target], c = 'Rating A$^c$ , B$^d$ , C$^e$')  

#hq_rabcd = hq_rabcd.rename(columns={target :target})

# Select rab rathings
hq_rabc = hq_no_pole.loc[hq_no_pole['code6'].isin(rabc)] # Only rab measurements
utils.hq_hist(hq_rabc, target, ax=ax[0], label = f'Rating A-C (N={len(hq_rabc):,})')
utils.print_latex_tab_line(d = hq_rabc[target], c = 'Rating A$^c$ , B$^d$ , C$^e$')  

#hq_rabc = hq_rabc.rename(columns={target :target})


# Select better ratings
hq_rab = hq_no_pole.loc[hq_no_pole['code6'].isin(rab)] # Only very rab measurements
utils.hq_hist(hq_rab, target,ax=ax[0],  label = f'Rating A-B (N={len(hq_rab):,})')
utils.print_latex_tab_line(d = hq_rab[target], c = 'Rating A$^c$ , B$^d$')    

#hq_rab = hq_rab.rename(columns={target :target})

# Select ra ratings
hq_ra = hq_no_pole.loc[hq_no_pole['code6'].isin(ra)] # Only the ra measurements
utils.hq_hist(hq_ra, target, ax=ax[0], label = f'Rating A (N={len(hq_ra):,})')
utils.print_latex_tab_line(d = hq_ra[target], c = 'Rating A$^c$')

#hq_ra = hq_ra.rename(columns={target :target})


for a in ax:
    a.legend()
    a.set_ylabel('N')

ax[1].set_xlabel('Elevation (m)')
ax[0].set_xlabel('Heat flow (mW/m2)')
    
ax[1].set_title('Cut-off elevation')
ax[0].set_title('Distribution of heat flow values after removal of records')
   
plt.show()
   

get index from grid and distance and transfer it to training data

In [ ]:
# transfer obs values from grid to hq integrate grid index


training_f = DIR /'Dataset'/'Preprocessed'

hq_ra_w = hq_ra[['lon', 'lat', 'GHF']].copy(deep=True)
hq_rab_w = hq_rab[['lon', 'lat', 'GHF']].copy(deep=True)


# volcanoes
geod = proj.Geod(ellps='WGS84' )
vol_int = volc_df[volc_df['lon'].between(world_lon_min, world_lon_max) 
                     & volc_df['lat'].between(world_lat_min, world_lat_max)]

#poistion of volvanoes inside targeted area
v_lons = vol_int['lon'].values.ravel()
v_lats = vol_int['lat'].values.ravel()

for hq,label in tqdm_notebook(
    zip([hq_ra_w, hq_rab_w], ['int_ra.csv','int_rab.csv']), 
                                          total=2, desc = 'Processing: '):

    sleep(0.01)

    model_coords = np.stack([ds_world.lon.values.ravel(), ds_world.lat.values.ravel()], axis=-1) # lat and lon for all cells ravelled
    max_dist = np.sum([spacing_world , spacing_world])/1.5 # equivalent to 15 km


    # stack coordin of best the good to compare to0 world grids[region]
    hq_coords = np.stack([hq['lon'], hq['lat']], axis=-1)
    # what is the index and distance of the neasrest points in world grids[region] to df
    grid_dists, indexs = spatial.KDTree(model_coords).query(hq_coords, 
                             distance_upper_bound=max_dist)
    hq.loc[:,'grid_index_world'] = indexs # This is the index to the cell in the ravel grids[region]
    hq.loc[:,'grid_index_world'] = hq.loc[:,'grid_index_world'].astype('int')

    dists_km = utils.distance(hq['lat'].values, hq['lon'].values, 
                    ds_world.lat.values.ravel()[indexs], ds_world.lon.values.ravel()[indexs])

    hq.loc[:,'dist_from_grid_world'] = dists_km*km


    for observable in list(ds_world):
            hq.loc[:,observable] = ds_world[observable].values.ravel()[hq.loc[:,'grid_index_world'].values] 
            
     
    ###volcanoes



    # this process is slow to map all grid

    volc_dist = []
    volc_dist_w = []
    # this process is slow to map all grid
    hq_lons, hq_lats = hq['lon'].values.ravel().reshape(-1,1) , hq['lat'].values.ravel().reshape(-1,1)
    for i in tqdm_notebook(range(len(hq)), desc='Volcs: ' ):
        sleep(0.01)
        hq_lat = np.repeat(hq_lats[i], len(v_lats) , axis=0).reshape(-1,1)
        hq_lon =np.repeat(hq_lons[i], len(v_lons) , axis=0).reshape(-1,1)
        _, _, ds = geod.inv(v_lons, v_lats, hq_lon, hq_lat)
        #idx = np.argpartition(ds, v_k)[:v_k]
        #ds_volcs[i] = (('Y', 'X'), ds.reshape(nn_world)/v_k/km)
        #volcs[i] = min(ds/v_k/km)
        volc_dist.append(min(ds)[0]/1/km)
        volc_dist_w.append(np.clip(1 - (min(ds)[0]/1/km)/volc_max_dist, 0, 1))
        #hq_volc.loc[i, 'VOLC_DIST_W'] = np.clip(1 - hq_volc.loc[i, 'VOLC_DIST']/volc_max_dist, 0, 1)
    hq['VOLC_DIST'] = volc_dist
    hq['VOLC_DIST_W'] = volc_dist_w



            
    hq.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
    hq.reset_index( inplace=True,drop=True)
    hq.to_csv(training_f/f'W_{label}' , index=False, header=True, sep='\t')


print('terminated')

In [ ]:
hq_ra_w = hq_ra[['lon', 'lat', 'GHF']].copy(deep=True)
hq_rab_w = hq_rab[['lon', 'lat', 'GHF']].copy(deep=True)



for hq,label in tqdm_notebook(
    zip([hq_ra_w, hq_rab_w], [f'int_ra.csv',f'int_rab.csv']), 
                                          total=2, desc = 'Processing: '):
    sleep(0.01)
    hq_afr = hq[(hq['lon'].between(afr_lon_min, afr_lon_max)  
                    & hq['lat'].between(afr_lat_min, afr_lat_max))].reset_index(drop=True)

    model_coords = np.stack([ds_afr.lon.values.ravel(), ds_afr.lat.values.ravel()], axis=-1) # lat and lon for all cells ravelled
    max_dist = np.sum([spacing_afr, spacing_afr])/1.5  # equivalent to 15 km


    # stack coordin of best the good to compare to0 world ds
    hq_coords = np.stack([hq_afr['lon'], hq_afr['lat']], axis=-1)
    # what is the index and distance of the neasrest points in world ds to df
    grid_dists, indexs = spatial.KDTree(model_coords).query(hq_coords, 
                             distance_upper_bound=max_dist)
    hq_afr.loc[:,'grid_index_afr'] = indexs # This is the index to the cell in the ravel ds

    hq_afr.loc[:,'grid_index_afr'] = hq_afr.loc[:,'grid_index_afr'].astype('int')

    dists_km = utils.distance( hq_afr['lat'].values, hq_afr['lon'].values,
                      ds_afr.lat.values.ravel()[indexs], ds_afr.lon.values.ravel()[indexs])

    hq_afr.loc[:,'dist_from_grid'] = dists_km*km

    for observable in list(ds_afr):
            hq_afr.loc[:,observable] = ds_afr[observable].values.ravel()[hq_afr.loc[:,'grid_index_afr'].values] 

    hq_afr = hq_afr.rename({target:'GHF'})
    hq_afr.sort_values(by=['lon', 'lat'], ascending=True, inplace=True)
    hq_afr.reset_index(drop=True, inplace=True)

    hq_afr.to_csv(training_f/f'Afr_{label}',  index=False, header=True, sep='\t')





print('terminated')


In [ ]:
ds_world.to_netcdf(DIR/'Grids'/'Inputs'/"ds_world.nc", mode='w', 
                    engine='netcdf4')

ds_afr.to_netcdf(DIR/'Grids'/'Inputs'/"ds_afr.nc", mode='w', 
                    engine='netcdf4')



#ds_world = xr.load_dataset(DIR/'Grids'/'Inputs'/"ds_world.nc")